In [2]:
from functools import reduce

from itertools import product

from datetime import datetime

import warnings

import pandas as pd

import numpy as np

from tableone import TableOne

import sklearn
from sklearn import preprocessing
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import ParameterGrid, cross_validate

from scipy.stats import ttest_rel

from pydtr.iqlearn.regression import IqLearnReg

import matplotlib.pyplot as plt

In [3]:
warnings.filterwarnings('ignore')

In [4]:
print(pd.__version__)

1.1.2


# Patient Characteristics (State)

In [ ]:
# import baseline and followup data
data = pd.read_csv('data/baseline_followup.csv')
data['VISIT DATE'] = pd.to_datetime(data['VISIT DATE'])
data.head()

In [ ]:
# Numerical variables
covariate_num = [
    'BMI',
    'Duration of diabetes',
    'HbA1c',
    'Age',
    'Family physican visits',
    'Family physican visits related to diabetes',
    'EQ5D',
    'AddQoL',
    'DSCA_general diet',
    'DSCA_specific diet',
    'DSCA_exercise',
    'DSCA_glucose',
    'DSCA_footcare',
    'DSCA_smoking2',
    'DSCA_additional diet',
    'DSCA_additional medication',
    'DSCA_additional footcare'
]

# Categorical variables
covariate_cat = [
    'Gender',
    'Ethnicity',
    'Diabetes treatment (diet)',
    'Diabetes treatment (oral therapy)',
    'Diabetes treatment (Insulin)',
    'Diabetes treatment (Other)',
    'Stroke',
    'Transient Ischemc Attack',
    'Evidence of CAD',
    'Myocardial infarction',
    'Heart Failure',
    'Kidney Disease',
    'COPD',
    'Hyperlipidemia',
    'Hypertension',
    'Peripheral Arterial Disease',
    'Previous limb amputation',
    'Prescribed medications', 
    'Behavioral stage',
    'Chronic disease mgmt program',
    'Visits with health professional',
    'ER/hospital admissions',
    'DSCA_smoking1',
]

In [ ]:
# Assess patient characteristics and generate Table 1
table1 = TableOne(data, columns = covariate_num+covariate_cat+['VISIT'], categorical = covariate_cat, groupby = 'VISIT', pval = True)
table1.to_csv('table1.csv')

# Outcome (Reward)

In [ ]:
# Relative HbA1c reduction and EQ5D improvement
for i in set(data['id']):
    # HbA1c
    data.loc[(data['id']==i)&(data['VISIT']==1), 'HbA1c_reduction_rel'] = (data[(data['id']==i)&(data['VISIT']==0)]['HbA1c'].values-data[(data['id']==i)&(data['VISIT']==1)]['HbA1c'].values)/data[(data['id']==i)&(data['VISIT']==0)]['HbA1c'].values
    data.loc[(data['id']==i)&(data['VISIT']==2), 'HbA1c_reduction_rel'] = (data[(data['id']==i)&(data['VISIT']==0)]['HbA1c'].values-data[(data['id']==i)&(data['VISIT']==2)]['HbA1c'].values)/data[(data['id']==i)&(data['VISIT']==0)]['HbA1c'].values
    # EQ5D
    data.loc[(data['id']==i)&(data['VISIT']==1), 'EQ5D_improve_rel'] = (data[(data['id']==i)&(data['VISIT']==1)]['EQ5D'].values-data[(data['id']==i)&(data['VISIT']==0)]['EQ5D'].values)/data[(data['id']==i)&(data['VISIT']==0)]['EQ5D'].values
    data.loc[(data['id']==i)&(data['VISIT']==2), 'EQ5D_improve_rel'] = (data[(data['id']==i)&(data['VISIT']==2)]['EQ5D'].values-data[(data['id']==i)&(data['VISIT']==0)]['EQ5D'].values)/data[(data['id']==i)&(data['VISIT']==0)]['EQ5D'].values

In [ ]:
# Derive the composite clinical outcome as relative HbA1c reduction and relative EQ5D improvement (normalized to [0,1], with a higher score being better)
scaler = preprocessing.MinMaxScaler()
data.loc[data['VISIT'].isin({1,2}), 'HbA1c_rel_scaled'] = scaler.fit_transform(data.loc[data['VISIT'].isin({1,2}), 'HbA1c_reduction_rel'].values.reshape(-1,1))
data.loc[data['VISIT'].isin({1,2}), 'EQ5D_rel_scaled'] = scaler.fit_transform(data.loc[data['VISIT'].isin({1,2}), 'EQ5D_improve_rel'].values.reshape(-1,1))

In [ ]:
data[data['VISIT']==1][['id', 'HbA1c_reduction_rel', 'EQ5D_improve_rel', 'HbA1c_rel_scaled', 'EQ5D_rel_scaled']].head()

In [ ]:
data[data['VISIT']==2][['id', 'HbA1c_reduction_rel', 'EQ5D_improve_rel', 'HbA1c_rel_scaled', 'EQ5D_rel_scaled']].head()

# Coaching (Action)

In [ ]:
# import coaching data
data_coaching = pd.read_csv('data/coaching.csv')
data_coaching['date of coaching'] = pd.to_datetime(data_coaching['date of coaching'])

In [ ]:
# Group coaching recommendations into three big groups of treatments
## treatment BC+DE (behavior change and diabetes education): Dietary modification, Exercise modification, Behavioural modification
## treatment CM (case management): Medication adherence, Medication adjustment, Glucose monitoring, Case-management/monitoring, System navigation
## treament PS (psychosocial support): Psychosocial support and/or counselling
data_coaching['treatment_BC+DE'] = data_coaching[[
        'Dietary modification', 
        'Exercise modification', 
        'Behavioural modification']].sum(axis = 1)
data_coaching['treatment_CM'] = data_coaching[[
        'Medication adherence', 
        'Medication adjustment', 
        'Glucose monitoring', 
        'Case-management/monitoring', 
        'System navigation']].sum(axis = 1)
data_coaching['treatment_PS'] = data_coaching[[
        'Psychosocial support and/or counselling']].sum(axis = 1)

In [ ]:
levels_recomm = [
    'Dietary modification',
    'Exercise modification',
    'Behavioural modification',
    'Medication adherence',
    'Medication adjustment',
    'Glucose monitoring',
    'Psychosocial support and/or counselling',
    'Case-management/monitoring',
    'System navigation'
    ]

levels_trt = [
    'treatment_BC+DE', 
    'treatment_CM', 
    'treatment_PS'
]

In [ ]:
# Map date of coaching with stages
## Stage 1: baseline to 6m follow-up visit
## Stage 2: 6m follow-up visit to 12m follow-up visit
for id in set(data['id']):

    # extract visit dates for baseline, 6m and 12m visits
    date_bl = data[(data['id']==id)&(data['VISIT']==0)]['VISIT DATE'].values[0]
    date_6m = data[(data['id']==id)&(data['VISIT']==1)]['VISIT DATE'].values[0]
    date_12m = data[(data['id']==id)&(data['VISIT']==2)]['VISIT DATE'].values[0]
    
    # Stage 1: baseline to 6m follow-up visit
    # Stage 2: 6m follow-up visit to 12m follow-up visit
    data_coaching.loc[(data_coaching['id']==id)&(data_coaching['date of coaching']>=date_bl)&(data_coaching['date of coaching']<date_6m), 'interval'] = 1
    data_coaching.loc[(data_coaching['id']==id)&(data_coaching['date of coaching']>=date_6m)&(data_coaching['date of coaching']<=date_12m), 'interval'] = 2
    data_coaching['interval'].fillna('out of bound', inplace = True)

print(data_coaching[['interval']].value_counts())

# Remove coaching data if the date of coaching is out of bound (i.e., not in stage 1 or stage 2)
data_coaching = data_coaching[data_coaching['interval']!='out of bound']

In [ ]:
# Feature engineering on coaching data
def coaching_fe(data, levels_recomm, levels_trt, interval):

    # Overall intensity: total # of coaching recommendation
    recomm_count = pd.DataFrame(data[levels_recomm+['id']].groupby('id').agg('sum')[levels_recomm].sum(axis=1))
    recomm_count.rename(columns = {recomm_count.columns[0]: 'recomm_count'}, inplace = True)
    
    # Relative intensity: proportion of each coaching recommendation
    data[levels_recomm] = data[levels_recomm].fillna(0)
    recomm = data[levels_recomm+['id']].groupby('id').agg('sum')
    recomm = recomm.merge(recomm_count, on = 'id')
    for col in levels_recomm:
        recomm[col] = recomm[col]/recomm['recomm_count']
        
    # Overall intensity: total # of treatment
    trt_count = pd.DataFrame(data[levels_trt+['id']].groupby('id').agg('sum')[levels_trt].sum(axis=1))
    trt_count.rename(columns = {trt_count.columns[0]: 'trt_count'}, inplace = True)

    # Relative intensity: proportion of each treatment
    data[levels_trt] = data[levels_trt].fillna(0)
    trt = data[levels_trt+['id']].groupby('id').agg('sum')
    trt = trt.merge(trt_count, on = 'id')
    for col in levels_trt:
        trt[col] = trt[col]/trt['trt_count']

    # Join recommendation and treatment
    coaching = reduce(lambda x, y: x.merge(y, on='id'), [recomm.fillna(0), trt.fillna(0)])
    coaching['interval'] = interval
    
    return coaching

In [ ]:
# Feature engineering on coaching data
data_coaching_s1 = coaching_fe(data_coaching[data_coaching['interval']==1], levels_recomm, levels_trt, interval=1)
data_coaching_s2 = coaching_fe(data_coaching[data_coaching['interval']==2], levels_recomm, levels_trt, interval=2)
coaching = pd.concat([data_coaching_s1, data_coaching_s2], axis = 0)

# Join patient characteristics (state) with coaching data (action)
data = data.merge(coaching, left_on = ['id', 'VISIT'], right_on = ['id', 'interval'], how = 'left')

# Fill missing values with 0
data = data.fillna(0)

In [ ]:
# Create the action space

## Intensity:
avg_intensity = np.median(data[data['VISIT'].isin([1, 2])]['trt_count']); print(avg_intensity)
data['intensity'] = np.where(data['trt_count'] > avg_intensity, 'high', 'low')

# Focus: coaching delivered in a stage can be classified as focused on one of the three treatments if a patient have at least twice as many treatments in one category compared to the others – otherwise they were classified as general coaching.
data['focus'] = np.where(data['treatment_BC+DE']/data['treatment_CM']>2, 'treatment_BC+DE',
                         np.where(data['treatment_CM']/data['treatment_BC+DE']>2, 'treatment_CM', 'treatment_mix'))

# Define an action using two dimensions: intensity and focus
data['treatment'] = data['intensity']+'_'+data['focus']

# Model Development and Validation

In [ ]:
# Specify the action variable
data = data.rename(columns = {'treatment':'action'})

In [ ]:
# Compute the composite clinical outcome variable as 0.5*relative reduction in HbA1c + 0.5**relative improvement in EQ5D
HbA1c_weight = 0.5
data['outcome'] = HbA1c_weight * data['HbA1c_rel_scaled'] + (1-HbA1c_weight) * data['EQ5D_rel_scaled']

In [ ]:
# Encode the action space
action_dict = {
    'low_treatment_BC+DE': 0, 'low_treatment_CM': 1, 'low_treatment_mix': 2,
    'high_treatment_BC+DE': 3, 'high_treatment_CM': 4, 'high_treatment_mix': 5
}

action_dict_inverse = {
    0: 'treatment_BC+DE', 1: 'treatment_CM', 2: 'treatment_mix',
    3: 'high intensity treatment_BC+DE', 4: 'high intensity treatment_CM', 5: 'high intensity treatment_mix'
}

data['action'] = data['action'].replace(action_dict)

In [ ]:
# Order-encoding the categorical variables
enc = preprocessing.OrdinalEncoder()
data[covariate_cat+['action']] = enc.fit_transform(data[covariate_cat+['action']])

In [ ]:
# Data preparation for the dynamic treatment regime model
predictors = covariate_num+covariate_cat

# State
X = data[data['VISIT'].isin([0, 1])][['id', 'VISIT'] + predictors]
X1 = X[X['VISIT']==0].drop(columns = ['VISIT'])
X2 = X[X['VISIT']==1].drop(columns = ['VISIT'])

# Reward
y1 = data[data['VISIT']==1][['outcome', 'id']]
y2 = data[data['VISIT']==2][['outcome', 'id']]

# Action
A1 = data[data['VISIT']==1][['action', 'id']]
A2 = data[data['VISIT']==2][['action', 'id']]

df1 = reduce(lambda x,y: pd.merge(x, y, on = 'id'), [X1, A1, y1])
df2 = reduce(lambda x,y: pd.merge(x, y, on = 'id'), [X2, A2, y2])

df1.columns = df1.columns+'_1'
df2.columns = df2.columns+'_2'

df = pd.merge(df1, df2, left_on = 'id_1', right_on = 'id_2').drop(columns = ['id_1', 'id_2'])
cols = df.columns.to_list()

cat_idx_stage1 = [df.columns.get_loc(c) for c in [col+'_1' for col in covariate_cat+['action']] if c in df]
cat_idx_stage2 = [df.columns.get_loc(c) for c in [col+'_1' for col in covariate_cat+['action']]+[col+'_2' for col in covariate_cat+['action']] if c in df]

In [ ]:
# Model development

# specify hyper-parameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.5], 
    'max_iter': [50, 100, 200, 1000], 
    'max_depth': [5, 10, None],
    'l2_regularization': [0.1, 0.5, 1]
}

params = [dict(zip(param_grid.keys(), value)) for value in product(*param_grid.values())]

# For loop for hyperpamameter tuning
outcomes_stage2 = []

for param in params:
    
    param1 = param.copy(); param1.update({'categorical_features': cat_idx_stage1})
    param2 = param.copy(); param2.update({'categorical_features': cat_idx_stage2})

    model_info = [
        {
            "model": HistGradientBoostingRegressor(**param1),
            "action_dict": {"action_1": [0, 1, 2, 3, 4, 5]},
            "feature": df1.drop(columns = ["id_1", "outcome_1"]).columns.to_list(),
            "outcome": "outcome_1",
            "importance": False
        },
        {
            "model": HistGradientBoostingRegressor(**param2),
            "action_dict": {"action_2": [0, 1, 2, 3, 4, 5]},
            "feature": df.drop(columns = ["outcome_2"]).columns.to_list(),
            "outcome": "outcome_2",
            "importance": False
        }
    ]

    # Fit model
    dtr_model = IqLearnReg(n_stages = 2, model_info = model_info)
    dtr_model.fit(df)
    
    # Compute the predicted end-of-stage 2 outcome
    res_stage2 =  dtr_model.predict(df, 1).rename(columns = {'val': 'val2'})
    outcomes_stage2.append(res_stage2['val2'].mean())

In [ ]:
# Select the hyperparameters associated the best predicted end-of-stage 2 outcome
param_selected = params[np.argmax(outcomes_stage2)]

param1 = param_selected.copy(); param1.update({'categorical_features': cat_idx_stage1})
param2 = param_selected.copy(); param2.update({'categorical_features': cat_idx_stage2})

model_info = [
    {
        "model": HistGradientBoostingRegressor(**param1),
        "action_dict": {"action_1": [0, 1, 2, 3, 4, 5]},
        "feature": df1.drop(columns = ["id_1", "outcome_1"]).columns.to_list(),
        "outcome": "outcome_1",
        "importance": False
    },
    {
        "model": HistGradientBoostingRegressor(**param2),
        "action_dict": {"action_2": [0, 1, 2, 3, 4, 5]},
        "feature": df.drop(columns = ["outcome_2"]).columns.to_list(),
        "outcome": "outcome_2",
        "importance": False
    }
]

In [ ]:
# Summarize the results

## Stage 1
res_stage1 = dtr_model.predict(df, 0).rename(columns = {'val': 'val1'})
## Stage 2
res_stage2 = dtr_model.predict(df, 1).rename(columns = {'val': 'val2'})
## Combine the results from stages 1 and 2
res = res_stage1.merge(res_stage2, how = 'inner', left_index = True, right_index = True)

# Calcualte the model's objective function value
res['objective_function'] = res['val1']
print('Model estimated (discounted) sum of reward:', res['objective_function'].mean(), sep = '')

In [ ]:
# Summarize the observed results

## Stage 1
y1 = y1.rename(columns = {'outcome': 'outcome1'})
## Stage 2
y2 = y2.rename(columns = {'outcome': 'outcome2'})
## Combine the observed results from stages 1 and 2
res_obs = y1.merge(y2, how = 'inner', left_on = 'id', right_on = 'id')

# Calcualte the observed objective function value
res_obs['objective_function'] = res_obs['outcome1'] + res_obs['outcome2']
print('Observed (discounted) sum of reward:', res_obs['objective_function'].mean())

In [ ]:
# Test for statistical significance (paired t test)
print(ttest_rel(res['objective_function'], res_obs['objective_function']))